In [14]:
from typing import Tuple, Optional
from data.hysteresis import hysteresis_ode
from data.utils import _load_wrapper
import numpy as np

from scipy.integrate import solve_ivp

In [16]:
def simulate_steady_state(
    lam_values, 
    x0, 
    t_span:Tuple[float, float],
    t_eval: Optional[np.ndarray] = None,
    show_progress: bool = True):
    """
    lam_values: control params 
    x0: initial state observation
    t_span: start and stop time of ode solution 
    t_eval: Points at which to save results. Default is end of t_span
    show_progress: Show progress bar (tqdm)
    
    """
    
    x_vals = []
    x_curr = x0

    wrapper = _load_wrapper(show_progress)

    if t_eval is None:
        t_eval = [t_span[-1]]
    
    for lam in wrapper(lam_values):
        sol = solve_ivp(hysteresis_ode, t_span, [x_curr], args=(lam,), t_eval=t_eval)
        x_curr = sol.y[0, -1]  # final value
        x_vals.append(x_curr)
        
    return np.array(x_vals)


In [ ]:
simulate_steady_state(